<!-- dom:TITLE: Partial differential equations and finite difference methods. -->
# Partial differential equations og finite difference methods.
<!-- dom:AUTHOR: Anne Kværnø, modified by André Massing, A. Schmeding -->
<!-- Author: -->  
**Anne Kværnø, modified by André Massing, A. Schmeding**

Date: **Jun 18, 2025**

La oss først laste inn moduler vi trenger

In [ ]:
%matplotlib inline

import numpy as np
from numpy import pi
from numpy.linalg import solve, norm    
import matplotlib.pyplot as plt

# Use a funny plotting style
plt.xkcd()
newparams = {'figure.figsize': (6.0, 6.0), 'axes.grid': True,
             'lines.markersize': 8, 'lines.linewidth': 2,
             'font.size': 14}
plt.rcParams.update(newparams)

# Innledning 

I dette notatet skal den endelige differansemetoden for å løse partialdifferensiallikninger (PDE-er) presenteres.
Grovsett består en endelig differansemetode av følgende trinn:

1. Diskretiser definisjonsmengden av likningen.

2. Erstatt de deriverte i hvert rutenettpunkt med en approksimasjon, ved å bruke verdiene i nabopunktene.

3. Erstatt den eksakte løsningen med dens approksimasjon.

4. Løs det resulterende ligningssystemet.

Vi skal først se hvordan man finner approksimasjoner til den deriverte av en funksjon, og deretter hvordan dette kan brukes til å løse randverdiproblemer (RVP) som

$$
u'' + p(x) u' + q(x) u = r(x), \qquad a \leq x \leq b, \qquad u(a)=u_a, \quad
u(b)=u_b,
$$

# Numerisk derivasjon
Dette er hovedverktøyet for endelige differansemetoder.

Gitt en tilstrekkelig glatt funksjon $f$. Hvordan kan vi finne en approksimasjon til $f'(x)$ eller $f''(x)$ i et gitt punkt $x$, kun ved å evaluere funksjonen selv?

[Den deriverte av $f$](https://wiki.math.ntnu.no/tma4100/tema/differentiation?&#definisjonen_av_den_deriverte_gitt_som_en_grenseverd)
er definert som

$$
f'(x) = \lim_{h\rightarrow 0} \frac{f(x+h)-f(x)}{h}.
$$

For liten $h$, høyre side av ligningen approksimerer $f'(x)$. De vanligste metoder for det er disse:

$$
f'(x) \approx \left\{
   \begin{array}{ll}
     \displaystyle \frac{f(x+h)-f(x)}{h}, \qquad & \text{Fremover differanse,} \\ 
     \displaystyle \frac{f(x)-f(x-h)}{h}, & \text{Bakover differanse,} \\ 
     \displaystyle \frac{f(x+h)-f(x-h)}{2h}, & \text{Sentral differanse.}
   \end{array} \right.
$$

$$
f''(x) \approx \frac{f(x+h)-2f(x)+f(x-h)}{h^2}.
$$

**Numerisk eksempel:**
Test metoden for $f(x)=\sin(x)$ ved $x=\frac{\pi}{4}$.
Samenlikne med den eksakte deriverte. Prøv ulike steg lengde, f.eks. $h=0.1, h=0.01, h=0.001$.
Se hvordan feilen endrer seg med $h$.

In [ ]:
from numpy import *               
from scipy.sparse import diags	        # Greate diagonal matrices
from scipy.linalg import solve	        # Solve linear systems
from matplotlib.pyplot import *     	
from mpl_toolkits.mplot3d import Axes3D  # For 3-d plot
from matplotlib import cm 
newparams = {'figure.figsize': (8.0, 4.0), 'axes.grid': True,
             'lines.markersize': 8, 'lines.linewidth': 2,
             'font.size': 14}
rcParams.update(newparams)
from numpy import *

In [ ]:
# Numerisk derivasjon

# Fremover differanse
def diff_forward(f, x, h=0.1):
    return (f(x+h)-f(x))/h

# Backlengs differanse
def diff_backward(f, x, h=0.1):
    return (f(x)-f(x-h))/h
 
# Sentral differanse for f'(x):
def diff_central(f, x, h=0.1):
    return (f(x+h)-f(x-h))/(2*h)
# end of diff_central

# Sentral differanse for f''(x):
def diff2_central(f, x, h=0.1):
    return (f(x+h)-2*f(x)+f(x-h))/h**2

In [ ]:
# Numerisk eksempel 1
x = pi/4;
df_exact = cos(x)
ddf_exact = -sin(x)
h = 0.1
f = sin
df = diff_forward(f, x, h)
print('Approximations to the first derivative')
print('Forward difference:  df = {:12.8f},   Error = {:10.3e} '.format(df, df_exact-df))
df = diff_backward(f, x, h)
print('Backward difference: df = {:12.8f},   Error = {:10.3e} '.format(df, df_exact-df))
df = diff_central(f, x, h)
print('Central difference:  df = {:12.8f},   Error = {:10.3e} '.format(df, df_exact-df))
print('Approximation to the second derivative') 
ddf = diff2_central(f, x, h)
print('Central difference:  ddf= {:12.8f},   Error = {:10.3e} '.format(ddf, ddf_exact-ddf))

## Bonus informasjon: Error analysis
In this case the error analysis is quite simple: Do a Taylor expansion of the
error around $x$. The Taylor expansion becomes a power series in $h$.


The expansion for the error of the forward difference is:

$$
e(x;h) = f'(x) - \frac{f(x+h)-f(x)}{h}  = f'(x) - \frac{f(x)+f'(x)h + \frac{1}{2}f''(\xi)h^2 - f(x)}{h} = -\frac{1}{2}f''(\xi)h
$$

where $\xi\in (x,x+h)$.  

The expansion for the error of the central difference is slightly more complicated:

$$
\begin{align*}
e(x; h) &= f'(x) - \frac{f(x+h)-f(x-h)}{2h} \\ 
        &= f'(x) \\ &- \frac{\big(f(x)+f'(x)h + \frac{1}{2} f''(x)h^2 + \frac{1}{6} f'''(\xi_1)h^2 \big) - \big(f(x)-f'(x)h + \frac{1}{2} f''(x)h^2 - \frac{1}{6} f'''(\xi_2)h^2\big)}{2h} \\ 
        &= -\frac{1}{12}\big(f'''(\xi_1) + f'''(\xi_2)\big)h^2  \\ 
        &= -\frac{1}{6}f'''(\eta)h^2, \qquad \qquad  \eta \in (x-h, x+h),
\end{align*}
$$

where the two remainder terms have been combined by the intermediate value theorem
(Result 2 at the end of *Preliminaries*). The error for the approximation of the
second order derivative can be found similarly. 

The order of an approximation is $p$ if there exist a constant $C$ independent on $h$ such that

$$
|e(h;x) \leq C h^p,
$$

In practice, it is sufficient to show that the power expansion of the error satisfies

$$
e(x,h)=C_ph^{p}+ C_{p+1}h^{p+1} + \dotsm, \qquad C_p \not=0
$$

The forward and backward approximations
are of order 1, the central differences of order 2. 

# To punkt randverdiproblemer (RVP)

Gitt en to punkt randverdiproblem:

$$
u'' + p(x) u' + q(x) u = r(x), \qquad a \leq x \leq b, \qquad u(a)=u_a, \quad u(b)=u_b,
$$

Der \(p\) og \(q\) er gitte funksjoner av \(x\), og randverdiene \(u_a\) og \(u_b\) er gitte konstanter.

En endelig differansemetode for dette problemet konstrueres ved følgende steg:

**Steg 1:**
Gitt intervallet \([a, b]\). Velg \(N\), la \(h = \frac{b - a}{N}\), og definer \(x_i = a + ih\), for \(i = 0, 1, \dotsc, N\).

**Steg 2:**
For hvert indre rutenettpunkt \(x_i\), \(i = 1, \dotsc, N-1\), erstatt de deriverte i RVP-en med deres approksimasjoner. Resultatet blir:


$$
\frac{u(x_i+h)-2u(x_i)+u(x_i-h)}{h^2} + p(x_i) \frac{u(x_i+h)-u(x_i-h)}{2h} + q(x_i) u(x_i) = r(x_i)
$$

**Steg 3:** Bytt eksakt løsning $u(x_i)$ med en numerisk (men fortsatt ukjent) approksimasjon $U_i$:

$$
\frac{U_{i+1}-2U_i+U_{i-1}}{h^2} + p(x_i)\frac{U_{i+1}-U_{i-1}}{2h} + q(x_i) U_i =  r(x_i), \qquad i=1,\dotsc,N-1.
$$

Dette er en *diskretisering* av RVPen. Hvis vi ha verdier på rand for ligningen, diskretisering er et lineært system

$$
A \mathbf{U} = \mathbf{b},
$$

hvor $A$ er en $N+1\times N+1$ matrise og  $\mathbf{U} = [U_0,\dotsc,U_{N}]^T$. Multipliserer viligninger med $h^2$ får vi:

$$
A =  \left[ \begin{array}{ccccccc}
      1 & 0 & &  \\ 
      v_1 & d_1 & w_1 & & &  \\ 
        & v_2 & d_2 & w_2 &  \\ 
        & & v_3 & \ddots & \ddots & \\ 
        & & & \ddots & \ddots & w_{N-2} \\ 
        & & & & v_{N-1} & d_{N-1} & w_{N-1}  \\ 
        & & & & &  0 & 1
    \end{array} \right]
    \qquad \text{med} \qquad
    \begin{array}{l}
    \displaystyle v_i =1-\frac{h}{2}p(x_i) \\ 
    \displaystyle d_i = -2 + h^2q(x_i) \\ 
    \displaystyle w_i = 1+\frac{h}{2}p(x_i)
    \end{array}.
$$

Høyre side $\mathbf{b}$ er gitt som

$$
\mathbf{b} = [u_a, h^2r(x_1), \dotsc, h^2r(x_{N-1}), u_b]^T.
$$

**Steg 4:** Løs  $A \mathbf{U} = \mathbf{b}$.


**Eksempel 1:**
Gitt ligningen 

$$
u'' + 2u' - 3u = 9x, \qquad u(0)=u_a = 1, \quad u(1)=u_b = e^{-3}+2e-5=0.486351,
$$

med eksakt løsning $u(x)= e^{-3x}+2e^{x}-3x-2$.

Velg $N$ og la $h=1/N$. Bruk sentrale differanser for $u'$ og $u''$, slik at

$$
\frac{u(x_i+h)-2u(x_i)+u(x_i-h)}{h^2} + 2 \frac{u(x_i+h)-u(x_i-h)}{2h} -3
u(x_i) =  9 x_i, \qquad i=1,\dotsc, N
$$

La $U_i \approx u(x_i)$. Multipliser med $h^2$ på begge sider og inkluder $U_0=u_a$ og $U_N=u_b$:

$$
\begin{align*}
 U_0 &= 1 \\ 
 (1-h)U_{i-1} + (-2-3h^2)U_i + (1+h)U_{i+1} &= 9x_ih^2, && i=1, \cdots N-1, \\ 
 U_N &= 0.486351
\end{align*}
$$

I praksis ha vi et system av 3 ligninger i 3 ukjente,

$$
\left(\begin{array}{ccc}
   -2.1875 & 1.25 & 0 \\ 
  0.75 & -2.1875 & 1.25 \\ 
   0 & 0.75 & -2.1875
\end{array} \right)
\left( \begin{array}{c} U_1 \\ U_2 \\ U_3  \end{array} \right) =
\left( \begin{array}{c} 0.140625-0.75\cdot 1 \\ 0.28125  \\  0.421875-1.25 \cdot
0.48635073  \end{array} \right),
$$

med løsning

$$
U_1 = 0.293176, \qquad U_2= 0.025557, \qquad  0.093820.
$$

Eksakte løsninger i punktene er:

$$
u(0.25) = 0.290417, \qquad u(0.5) = 0.020573, \qquad u(0.75) = 0.089400.
$$

## Implementering
For enkelhets skyld er implementeringen nedenfor kun gjort for randverdiproblemer (BVP-er) med konstante koeffisienter, det vil si at $p(x)=p$ og $q(x)=q$. Dette gjør at diagonalen, underdiagonalen og overdiagonalen i matrisen blir konstante, bortsett fra i første og siste rad.
En hjelpefunksjon er inkludert for å konstruere matriser av formen $A =
\text{tridiag}\{v,d,w\}$. 

Implementering består av
1. Velg $N$, la $h=(b-a)/N$ og $x_i=a+ih$, $i=0,\dotsc,N$.

2. Konstruer matrise $A\in \mathbb{R}^{N+1\times N+1}$ og vektor $b\in\mathbb{R}^{N+1}$. Matrisa $A$ er tridiagonal, med elementer (med unntak av første og siste rad) $v=1-\frac{h}{2}p$ under diagonalen, $d = -2 + h^2 q$ på diagonalen og  $w = 1+\frac{h}{2}p$ over diagonalen. 

3. Konstruer $\mathbf{b} = [b_0,\dotsc,b_N]^T$ med elemeneter $b_i=h^2r(x_i)$ for $i=1,\dotsc,N-1$.

4. Modifiser første og siste rad av $A$ og første og siste element i $\mathbf{b}$, med hensyn til randverdier

5. Løs $A\mathbf{U} = \mathbf{b}$.

In [ ]:
def tridiag(v, d, w, N):
    # Help function 
    # Returns a tridiagonal matrix A=tridiag(v, d, w) of dimension N x N.
    e = ones(N)        # array [1,1,...,1] of length N
    A = v*diag(e[1:],-1)+d*diag(e)+w*diag(e[1:],1)
    return A

In [ ]:
# Eksempel 1, RVP

# Define the equation 
# u'' + p*u' + q*u = r(x) on the interval [a,b]
# Boundary condition: u(a)=ua and u(b)=ub

p = 2
q = -3
def r(x):
    return 9*x
a, b = 0, 1
ua, ub = 1, exp(-3)+2*exp(1)-5

# The exact solution (if known)
def u_eksakt(x):
    return exp(-3*x)+2*exp(x)-3*x-2

# Set up the discrete system
N = 4                      # Number of intervals                  

# Start the discretization  
h = (b-a)/N                # Stepsize
x = linspace(a, b, N+1)    # The gridpoints x_0=a, x_1=a+h, .... , x_N=b 

# Make a draft of the A-matrix (first and last row have to be adjusted)
v = 1-0.5*h*p              # Subdiagonal
d = -2+h**2*q              # Diagonal
w = 1+0.5*h*p              # Superdiagonal
A = tridiag(v, d, w, N+1)  

# Make a draft of the b-vector
b = h**2*r(x)  

# Modify the first equation (left boundary) 
A[0,0] = 1
A[0,1] = 0
b[0] = ua
        
# Modify the last equation (right boundary)   
A[N,N] = 1              
A[N,N-1] = 0
b[N] = ub


U = solve(A, b)     #  Solve the equation

La oss verifisere beregninger. Print $A$, $\mathbf{b}$ og numerisk løsning $\mathbf{U}$.

In [ ]:
# Print the matrix A, the right hand side b the numerical and exact solution
print('A =\n', A)                 
print('\nb =\n ', b)
print('\nU =\n ', U)
print('\nu(x)=\n', u_eksakt(x))

In [ ]:
# Plot the solution of the BVP
xe = linspace(0,1,101)
plot(x,U,'.-')
plot(xe, u_eksakt(xe),':')              
xlabel('x')
ylabel('u')
legend(['Numerisk','Eksakt'])
title('Løsningen av et to-punkt randverdiproblem.');

In [ ]:
# Plot the error |u(x)-U| in the gridpoints
error = abs(u_eksakt(x)-U)
plot(x, error,'.-')
xlabel('x')
ylabel('error')
title('Error: u(x)-U');
print('Max error = {:.3e}'.format(max(abs(error))))

** Eksempel 2:**

Gitt det samme eksempelet som før, men nå med en Neumann-betingelse i venstre randpunkt:

$$
u'' + 2u' - 3u = 9x, \qquad u'(0)=u'_a =-4, \quad u(1)=u_b = -2e^{-3}+e-5 =
0.48635073,
$$

med eksakt løsning $u(x)= e^{-3x}-2e^{x}-3x-2$.

Modifisert ligning ved rand $x_0=0$ er:

$$
\frac{2U_1-2U_0-2u'_a h}{h^2} +2u'_a - 3U_0 = 0.
$$

Multipliser med $h^2$, og inkluder ligning som diskretisering for $x_0$.

$$
\begin{align*}
 (-2-3h^2)U_0 - 2U_1 &= (2h-2h^2)u'_a \\ 
 (1-h)U_{i-1} + (-2-3h^2)U_i + (1+h)U_{i+1} &= 9 h^2 x_i, && i=1, \cdots N-1. \\ 
 U_N &= u_b,
\end{align*}
$$

for $N=4$ og $h=0.25$ er det:

$$
\left( \begin{array}{ccccc}
  -2.1875 & 2 & 0 & 0 & 0  \\ 
  0.75 & -2.1875 & 1.25 & 0 & 0 \\ 
 0 & 0.75 & -2.1875 & 1.25 & 0 \\ 
 0 & 0 & 0.75 & -2.1875 & 1.25 \\ 
 0 & 0 & 0 & 0 & 1 \\ 
\end{array}\right)\left(\begin{array}{c} U_{0} \\ U_1 \\ U_2 \\ U_3 \\ U_4
\end{array}\right) = \left(\begin{array}{c} -1.5  \\ 0.140625  \\  0.28125  \\ 
0.421875  \\  0.48635073 \end{array} \right).
$$

Løsnignen er

$$
U_0 = 0.92103219, \quad U_1 = 0.25737896, \quad  U_2 = 0.01029386, \qquad U_3 = 0.08858688.
$$

**Numerisk Øving:**
1. Modifiser koden over for å løse problemet. Bruk $N=4$ for å sjekke løsningen, men også prøv $N=10$ og $N=20$. 

2.  Modifiser koden over for å løse RVP, men nå med venstre rand betingelse \\ $u'(a)+u(a)/4=0$. 